In [1]:
!pip install -U adapter-transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
dataset.num_rows


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': 8530, 'validation': 1066, 'test': 1066}

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [4]:
dataset.num_rows

{'train': 8530, 'validation': 1066, 'test': 1066}

In [5]:
dataset["train"][0]

{'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'label': 1}

In [6]:
dataset["test"][0]

{'text': 'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .',
 'label': 1}

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

In [8]:
from transformers import AutoTokenizer, RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [9]:
def encode_batch(batch):
  return tokenizer(batch["text"], max_length=100, truncation = True, padding="max_length")

In [10]:
dataset = dataset.map(encode_batch, batched=True)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [11]:
dataset = dataset.rename_column("label", "labels")

In [12]:
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [13]:
from transformers import RobertaConfig, RobertaModelWithHeads
model = RobertaModelWithHeads.from_pretrained("roberta-base")

/usr/local/lib/python3.9/dist-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


/usr/local/lib/python3.9/dist-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [14]:
model.add_adapter("rotten_tomatoes")
# Add a matching classification head
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter("rotten_tomatoes")

In [25]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=3e-4,
    max_steps=30000,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=1000,
    output_dir="adapter-roberta-base-rotten_tomatoes-v1",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(eval_pred):
  preds = np.argmax(eval_pred.predictions, axis=1)
  return {"acc": (preds == eval_pred.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 8530
  Num Epochs = 113
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 30000
  Number of trainable parameters = 1486658


Step,Training Loss
1000,0.291800
2000,0.143400
3000,0.075100
4000,0.047700
5000,0.030400
6000,0.030700
7000,0.019700
8000,0.021500
9000,0.018300
10000,0.014900


Saving model checkpoint to adapter-roberta-base-rotten_tomatoes/checkpoint-500
Configuration saved in adapter-roberta-base-rotten_tomatoes/checkpoint-500/rotten_tomatoes/adapter_config.json
Module weights saved in adapter-roberta-base-rotten_tomatoes/checkpoint-500/rotten_tomatoes/pytorch_adapter.bin
Configuration saved in adapter-roberta-base-rotten_tomatoes/checkpoint-500/rotten_tomatoes/head_config.json
Module weights saved in adapter-roberta-base-rotten_tomatoes/checkpoint-500/rotten_tomatoes/pytorch_model_head.bin
Configuration saved in adapter-roberta-base-rotten_tomatoes/checkpoint-500/rotten_tomatoes/head_config.json
Module weights saved in adapter-roberta-base-rotten_tomatoes/checkpoint-500/rotten_tomatoes/pytorch_model_head.bin
Saving model checkpoint to adapter-roberta-base-rotten_tomatoes/checkpoint-1000
Configuration saved in adapter-roberta-base-rotten_tomatoes/checkpoint-1000/rotten_tomatoes/adapter_config.json
Module weights saved in adapter-roberta-base-rotten_tomatoes

TrainOutput(global_step=30000, training_loss=0.026443682020902633, metrics={'train_runtime': 2962.6037, 'train_samples_per_second': 324.039, 'train_steps_per_second': 10.126, 'total_flos': 5.01067770384e+16, 'train_loss': 0.026443682020902633, 'epoch': 112.36})

In [17]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1066
  Batch size = 32


{'eval_loss': 1.1206598281860352,
 'eval_acc': 0.8893058161350844,
 'eval_runtime': 1.5378,
 'eval_samples_per_second': 693.205,
 'eval_steps_per_second': 22.11,
 'epoch': 112.36}

In [18]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("I came across a lot of reviews stating that it is the best book out there.")

The model 'RobertaModelWithHeads' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'EsmForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification', 'GPTNeoForSequenceClassification', 'GPTJForSequenceClassification', 'IBertForSequenceClassification', 'LayoutLMForSequenceClassification'

[{'label': '👍', 'score': 0.9999892711639404}]

In [19]:
model.save_adapter("./final_adapter", "rotten_tomatoes")



Configuration saved in ./final_adapter/adapter_config.json
Module weights saved in ./final_adapter/pytorch_adapter.bin
Configuration saved in ./final_adapter/head_config.json
Module weights saved in ./final_adapter/pytorch_model_head.bin


In [20]:
!ls -lh final_adapter

total 5.7M
-rw-r--r-- 1 root root 1.1K Mar 17 00:22 adapter_config.json
-rw-r--r-- 1 root root  419 Mar 17 00:22 head_config.json
-rw-r--r-- 1 root root 3.5M Mar 17 00:22 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Mar 17 00:22 pytorch_model_head.bin


In [22]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("my-awesome-model")

In [ ]:
!pip install pickle5

In [ ]:
from huggingface_hub import HfApi, snapshot_download

from huggingface_hub.repocard import metadata_eval_result, metadata_save

from pathlib import Path
import pickle5 as pickle
import datetime
import json

In [ ]:
def push_to_hub(repo_id, model, local_repo_path="hub"):
    """
    Evaluate, Generate a video and Upload a model to Hugging Face Hub.
    This method does the complete pipeline:
    - It evaluates the model
    - It generates the model card
    - It generates a replay video of the agent
    - It pushes everything to the Hub

    :param repo_id: repo_id: id of the model repository from the Hugging Face Hub
    :param env
    :param video_fps: how many frame per seconds to record our video replay
    (with taxi-v3 and frozenlake-v1 we use 1)
    :param local_repo_path: where the local repository is
    """
    _, repo_name = repo_id.split("/")

    api = HfApi()

    # Step 1: Create the repo
    repo_url = api.create_repo(
        repo_id=repo_id,
        exist_ok=True,
    )

    # Step 2: Download files
    repo_local_path = Path(snapshot_download(repo_id=repo_id))


    # Pickle the model
    with open((repo_local_path) / "adapter-roberta-base-rotten-tomatoes.pkl", "wb") as f:
        pickle.dump(model, f)


    evaluate_data = {
        "eval_loss": 0.1150890588760376,
        'eval_acc': 0.95941,
        'eval_samples_per_second': 715.503,
        'eval_steps_per_second': 22.359,
        'eval_steps_per_second': 22.359,
        'epoch': 0.71,
        "eval_datetime": datetime.datetime.now().isoformat(),
    }


    # Write a JSON file called "results.json" that will contain the
    # evaluation results
    with open(repo_local_path / "results.json", "w") as outfile:
        json.dump(evaluate_data, outfile)



    metadata = {}
    metadata["tags"] = ["text-classification", "amazon-polarity", "roberta-base", "Adapter"]


    # Merges both dictionaries
    metadata = {**metadata}

    model_card = f"""
    # **we train an adapter for a RoBERTa model for sequence classification on a sentiment analysis task using adapter-transformers, the AdapterHub adaptation of HuggingFace's transformers library.**

    ## Usage

    ```python

    model = load_from_hub(repo_id="{repo_id}", filename="adapter-roberta-base-amazon-polarity.pkl")

    ```
    """



    readme_path = repo_local_path / "README.md"
    readme = ""
    print(readme_path.exists())
    if readme_path.exists():
        with readme_path.open("r", encoding="utf8") as f:
            readme = f.read()
    else:
        readme = model_card

    with readme_path.open("w", encoding="utf-8") as f:
        f.write(readme)

    # Save our metrics to Readme metadata
    metadata_save(readme_path, metadata)


    # Step 7. Push everything to the Hub
    api.upload_folder(
        repo_id=repo_id,
        folder_path=repo_local_path,
        path_in_repo=".",
    )

    print("Your model is pushed to the Hub. You can view your model here: ", repo_url)

In [ ]:
username = "DrishtiSharma"  # FILL THIS
repo_name = "Adapter-for-RoBERTa-on-rotten-tomatoes"
push_to_hub(repo_id=f"{username}/{repo_name}", model=model)

In [21]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [23]:
!git config --global credential.helper store

In [26]:
trainer.push_to_hub()

Cloning https://huggingface.co/DrishtiSharma/adapter-roberta-base-rotten_tomatoes-v1 into local empty directory.
Saving model checkpoint to adapter-roberta-base-rotten_tomatoes-v1
Configuration saved in adapter-roberta-base-rotten_tomatoes-v1/rotten_tomatoes/adapter_config.json
Module weights saved in adapter-roberta-base-rotten_tomatoes-v1/rotten_tomatoes/pytorch_adapter.bin
Configuration saved in adapter-roberta-base-rotten_tomatoes-v1/rotten_tomatoes/head_config.json
Module weights saved in adapter-roberta-base-rotten_tomatoes-v1/rotten_tomatoes/pytorch_model_head.bin
Configuration saved in adapter-roberta-base-rotten_tomatoes-v1/rotten_tomatoes/head_config.json
Module weights saved in adapter-roberta-base-rotten_tomatoes-v1/rotten_tomatoes/pytorch_model_head.bin


Upload file rotten_tomatoes/pytorch_adapter.bin:   1%|          | 32.0k/3.43M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.43k/3.43k [00:00<?, ?B/s]

Upload file rotten_tomatoes/pytorch_model_head.bin:   1%|1         | 32.0k/2.26M [00:00<?, ?B/s]

To https://huggingface.co/DrishtiSharma/adapter-roberta-base-rotten_tomatoes-v1
   1444330..97ca92e  main -> main

   1444330..97ca92e  main -> main

Dropping the following result as it does not have all the necessary fields:
{'dataset': {'name': 'rotten_tomatoes', 'type': 'rotten_tomatoes', 'config': 'default', 'split': 'validation', 'args': 'default'}}
To https://huggingface.co/DrishtiSharma/adapter-roberta-base-rotten_tomatoes-v1
   97ca92e..81dc123  main -> main

   97ca92e..81dc123  main -> main



'https://huggingface.co/DrishtiSharma/adapter-roberta-base-rotten_tomatoes-v1/commit/97ca92e2dd4c43db3318645dd02c193c1126c5e1'